In [1]:
#Needed in general
from __future__ import division

import numpy as np
import matplotlib.pyplot as plt
import sys
import h5py as h5
import gc

#Quick fudge to make import from ../Scripts work
sys.path.append('../../Scripts')

#Custom imports
import ClassCOMPAS     as CC ###
# import ClassFormationChannels as FC a


# import script that has formation channel classification functions:
from PostProcessingScripts import * 
from ClassFormationChannels_5mainchannels import * 



from astropy import units as u
from astropy import constants as const

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
print(fs)

import astropy
import seaborn as sns
# from seaborn_utility import cmap_from_colour

24


In [2]:
def function_f_Bavera21(m_WR, c1, c2, c3):
    """
    m_WR with units using astropy
    
    
    """
    
    
    
    top = -c1
    bottom = c2 + np.exp(-c3*m_WR.to(u.Msun).value)
    
    f = top/bottom

    
    return f 



def calculate_alpha_beta_Bavera21(m_WR, c1_alpha, c2_alpha, c3_alpha,  c1_beta,  c2_beta,  c3_beta):
    
    
    alpha = function_f_Bavera21(m_WR, c1_alpha, c2_alpha, c3_alpha)
    beta  = function_f_Bavera21(m_WR, c1_beta,  c2_beta,  c3_beta)
    
    return alpha, beta


def calculate_BH_spin_Bavera21(m_WR, period, maskBH2):
    """
    based on Eq 1 and 2 from https://arxiv.org/pdf/2105.09077.pdf
    
    
    """
 
    # numerical coefficients form text below Eq 2
    # we use the values at helium depletion, since we later on use the C/O core mass. 
    c1_alpha, c2_alpha, c3_alpha =  0.059305, 0.035552, 0.270245
    c1_beta,  c2_beta, c3_beta   =  0.026960, 0.011001, 0.420739
    
    alpha, beta = calculate_alpha_beta_Bavera21(m_WR, c1_alpha, c2_alpha, c3_alpha,  c1_beta,  c2_beta,  c3_beta)
    print(alpha, beta)
    
    
    # make array with zeroes with length similar to input
    print(m_WR)
    a_BH2 = np.zeros(len(m_WR.value))

    
    print(period.value)
    maskBH2_and_P_below_1Day = (maskBH2==1) & (np.asarray(period.value) <= 1)
    print(maskBH2_and_P_below_1Day)
    
    
    
    # if they formed second, update the BH spin with the Bavera spin 
    print('inside function', len(period), len(maskBH2_and_P_below_1Day), len(period[maskBH2_and_P_below_1Day].to(u.d).value))
    
#     print((len(alpha) * (np.log10(period[maskBH2_and_P_below_1Day].to(u.d).value))**2))
    first_term = (alpha[maskBH2_and_P_below_1Day] * (np.log10(period[maskBH2_and_P_below_1Day].to(u.d).value))**2) 
#     print('first = ', first_term)
#     print(period[maskBH2_and_P_below_1Day].to(u.d).value)
    second_term =  ( beta[maskBH2_and_P_below_1Day] * np.log10(period[maskBH2_and_P_below_1Day].to(u.d).value))  
    
#     print(len(first_term))
#     print(len(second_term))
#     print(len( first_term  + second_term))
#     print(a_BH2[maskBH2_and_P_below_1Day])
    a_BH2[maskBH2_and_P_below_1Day]  =  first_term  + second_term  
    
    return a_BH2




    
    
    
    
    

In [3]:
# print(calculate_BH_spin_Bavera21(m_WR=np.asarray([2, 2])*u.Msun, period=np.asarray([0.5, 0.5])*u.d, maskBH2=np.asarray([True, True])))

In [4]:
# BPSmodelName='P'
# mssfr = '112'
# DCOtype='BHNS'
# # path for files 
# path_ = '/Volumes/Andromeda/DATA/AllDCO_bugfix/' + alphabetDirDict[BPSmodelName] +'/'
# path  = path_ + 'COMPASCompactOutput_'+ DCOtype +'_' + BPSmodelName + '.h5'

# # read in data 
# fdata = h5.File(path)


# # M1 will be the most massive, M2 the least massive compact object. 
# M1, M2 = obtainM1BHandM2BHassymetric(m1=fdata['doubleCompactObjects']['M1'][...].squeeze(), m2=fdata['doubleCompactObjects']['M2'][...].squeeze()) 
# Chirpmass = chirpmass(M1, M2)
# MassRatio = M2/M1

# ### read in MSSFR weights: ###

# # get detected weights
# fparam_key = 'weights_detected'
# weightheader = 'w_' + mssfr
# w = fdata[fparam_key][weightheader][...].squeeze()

# labellist=[ r'$\rm{m}_{\rm{chirp}} \ [\rm{M}_{\odot}]$', r'$\rm{m}_{\rm{BH}} \ [\rm{M}_{\odot}]$', r'$\rm{m}_{\rm{NS}} \ [\rm{M}_{\odot}]$',\
#        r'$q $'] #,  r'$\log_{10}(t) $']              
# varlist = [Chirpmass, M1, M2, MassRatio]


# levels = [0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]
# level_labels = [r"$98\%$", r"$90\%$", r"$75\%$", r"$50\%$", r"$25\%$", r"$10\%$", r"$0\%$"]



# maskGW200105  = ((M1 <= (8.9+1.2))  & (M1>=(8.9-1.5))) & ((M2 <= (1.9+0.3))  & (M2>=(1.9-0.2))) & ((chirpmass(M1,M2)>=(3.41-0.07)) & (chirpmass(M1,M2)<=(3.41+0.08))) & ((M2/M1>=(0.26-0.1)) & (M2/M1<=(0.26+0.35)))
# maskGW200115  = ((M1 <= (5.7+1.8))  & (M1>=(5.7-2.1))) & ((M2 <= (1.5+0.7))  & (M2>=(1.5-0.3))) & ((chirpmass(M1,M2)>=(2.42-0.07)) & (chirpmass(M1,M2)<=(2.42+0.05))) & ((M2/M1>=(0.26-0.1)) & (M2/M1<=(0.26+0.35))) 


# # data needed to calculate spin 
# seedsDCO = fdata['doubleCompactObjects']['seed'][...].squeeze()
# seedsSN = fdata['supernovae']['randomSeed'][...].squeeze()



# print(fdata['supernovae'].keys())

# print(fdata['supernovae']['MassCOCoreSN'][...].squeeze()[0:10])
# print(fdata['supernovae']['MassCoreSN'][...].squeeze()[0:10])

# print(fdata['supernovae']['MassStarSN'][...].squeeze()[0:10])




In [5]:
print(1*u.Msun)

1.0 solMass


In [6]:
def print_spinning_BHNS(BPSmodelName='P', mssfr = '112'):
    """
    
    mssfr = '112' # mssfr that matches LIGO observed rates.
    """
    
    
    DCOtype='BHNS'
    # path for files 
    path_ = '/Volumes/Andromeda/DATA/AllDCO_bugfix/' + alphabetDirDict[BPSmodelName] +'/'
    path  = path_ + 'COMPASCompactOutput_'+ DCOtype +'_' + BPSmodelName + '.h5'

    # read in data 
    fdata = h5.File(path)


    # M1 will be the most massive, M2 the least massive compact object. 
    M1, M2 = obtainM1BHandM2BHassymetric(m1=fdata['doubleCompactObjects']['M1'][...].squeeze(), m2=fdata['doubleCompactObjects']['M2'][...].squeeze()) 
    Chirpmass = chirpmass(M1, M2)
    MassRatio = M2/M1

    ### read in MSSFR weights: ###
    
    # get detected weights
    fparam_key = 'weights_detected'
    weightheader = 'w_' + mssfr
    w = fdata[fparam_key][weightheader][...].squeeze()

    labellist=[ r'$\rm{m}_{\rm{chirp}} \ [\rm{M}_{\odot}]$', r'$\rm{m}_{\rm{BH}} \ [\rm{M}_{\odot}]$', r'$\rm{m}_{\rm{NS}} \ [\rm{M}_{\odot}]$',\
           r'$q $'] #,  r'$\log_{10}(t) $']              
    varlist = [Chirpmass, M1, M2, MassRatio]


    levels = [0.02, 0.1, 0.25, 0.5, 0.75, 0.9, 1.0]
    level_labels = [r"$98\%$", r"$90\%$", r"$75\%$", r"$50\%$", r"$25\%$", r"$10\%$", r"$0\%$"]



    maskGW200105  = ((M1 <= (8.9+1.2))  & (M1>=(8.9-1.5))) & ((M2 <= (1.9+0.3))  & (M2>=(1.9-0.2))) & ((chirpmass(M1,M2)>=(3.41-0.07)) & (chirpmass(M1,M2)<=(3.41+0.08))) & ((M2/M1>=(0.26-0.1)) & (M2/M1<=(0.26+0.35)))
    maskGW200115  = ((M1 <= (5.7+1.8))  & (M1>=(5.7-2.1))) & ((M2 <= (1.5+0.7))  & (M2>=(1.5-0.3))) & ((chirpmass(M1,M2)>=(2.42-0.07)) & (chirpmass(M1,M2)<=(2.42+0.05))) & ((M2/M1>=(0.26-0.1)) & (M2/M1<=(0.26+0.35))) 

    
    # data needed to calculate spin 
    seedsDCO = fdata['doubleCompactObjects']['seed'][...].squeeze()
    seedsSN = fdata['supernovae']['randomSeed'][...].squeeze()
    # get only SN seeds for DCOs 
    maskSNdco = np.in1d(seedsSN,  seedsDCO) 
    whichSN = fdata['supernovae']['whichStar'][...].squeeze()[maskSNdco]
    whichSN1 = whichSN[::2] # get whichStar for first SN 

    separationPreSN = fdata['supernovae']['separationBefore'][...].squeeze()[maskSNdco]
    separationPreSN2 = separationPreSN[1::2] # in Rsun. 
    maskNSBH = ((whichSN1==2) & (M1>M2) ) | ((whichSN1==1) & (M1<M2) )      


    # get the MassCOCoreSN for the second SN
    MassCOCoreSN = fdata['supernovae']['MassCOCoreSN'][...].squeeze()[1::2] 
    
    
    
    

    

    
    
    print('----- OLD OLD AND WRONG my fault analytical fit Debatri ----- ')

    
    # M1 is compact object mass, so the same 
    spinBH = QinBHspinmodel(separationPreSN2, M1, M2, maskNSBH)  
    mask_spin = (spinBH >=0.05)
    mask_spinBig = (spinBH>=0.2)
    
    print('--- small >0.05 BH spin ---')
    print('fraction of all BHNS = ', np.sum(w[mask_spin])/(np.sum(w)) )
    print('fraction of all GW200115 = ', np.sum(w[(mask_spin==1) & (maskGW200115==1)])/np.sum(w[maskGW200115]) )
    print('fraction of all GW200105 = ', np.sum(w[(mask_spin==1) & (maskGW200105==1)])/np.sum(w[maskGW200105]) )
    
    print()
    

    print('--- moderate >0.2 BH spin---')
    print('fraction of all BHNS = ', np.sum(w[mask_spinBig])/(np.sum(w)) )
    print('fraction of all GW200115 = ', np.sum(w[(mask_spinBig==1) & (maskGW200115==1)])/np.sum(w[maskGW200115]) )
    print('fraction of all GW200105 = ', np.sum(w[(mask_spinBig==1) & (maskGW200105==1)])/np.sum(w[maskGW200105]) )
    
    print()
    
    
    
    
    print('----- old analytical fit Debatri ----- ')
    
    # M1 is compact object mass, so the same 
    spinBH = QinBHspinmodel(separationPreSN2, M1, MassCOCoreSN, maskNSBH)  
    mask_spin = (spinBH >=0.05)
    mask_spinBig = (spinBH>=0.2)
    
    
    print('--- small >0.05 BH spin ---')
    print('fraction of all BHNS = ', np.sum(w[mask_spin])/(np.sum(w)) )
    print('fraction of all GW200115 = ', np.sum(w[(mask_spin==1) & (maskGW200115==1)])/np.sum(w[maskGW200115]) )
    print('fraction of all GW200105 = ', np.sum(w[(mask_spin==1) & (maskGW200105==1)])/np.sum(w[maskGW200105]) )
    
    print()
    

    print('--- moderate >0.2 BH spin---')
    print('fraction of all BHNS = ', np.sum(w[mask_spinBig])/(np.sum(w)) )
    print('fraction of all GW200115 = ', np.sum(w[(mask_spinBig==1) & (maskGW200115==1)])/np.sum(w[maskGW200115]) )
    print('fraction of all GW200105 = ', np.sum(w[(mask_spinBig==1) & (maskGW200105==1)])/np.sum(w[maskGW200105]) )
    
    print()
    
    print('------ Bavera 2021 ------- ')
    print()
    
    # M1 is compact object, M2 is WR mass 
    M2_WR = np.asarray(MassCOCoreSN) * u.Msun 
    Period = convert_a_to_P_circular(separation=separationPreSN2*u.Rsun, M1=M1*u.Msun, M2=M2_WR)
    
    print(len(maskNSBH), len(MassCOCoreSN), len(Period))
    spinBH2_Bavera21 = calculate_BH_spin_Bavera21(m_WR=M2_WR, period=Period.to(u.d), maskBH2=maskNSBH)
    mask_spin = (spinBH2_Bavera21 >=0.05)
    mask_spinBig = (spinBH2_Bavera21>=0.2)    

    
    print('--- small >0.05 BH spin ---')
    print('fraction of NS--BH All BHNS = ', np.sum(w[maskNSBH])/(np.sum(w)) )
    print('fraction of all BHNS = ', np.sum(w[mask_spin])/(np.sum(w)) )
    
    print('fraction of all GW200115 NS-BH = ', np.sum(w[(maskNSBH==1) & (maskGW200115==1)])/np.sum(w[maskGW200115]) )
    print('fraction of all GW200105 NS-BH = ', np.sum(w[(maskNSBH==1) & (maskGW200105==1)])/np.sum(w[maskGW200105]) )
    print('fraction of all GW200115 = ', np.sum(w[(mask_spin==1) & (maskGW200115==1)])/np.sum(w[maskGW200115]) )
    print('fraction of all GW200105 = ', np.sum(w[(mask_spin==1) & (maskGW200105==1)])/np.sum(w[maskGW200105]) )
    
    print()
    

    print('--- moderate >0.2 BH spin---')
    print('fraction of all BHNS = ', np.sum(w[mask_spinBig])/(np.sum(w)) )
    print('fraction of all GW200115 = ', np.sum(w[(mask_spinBig==1) & (maskGW200115==1)])/np.sum(w[maskGW200115]) )
    print('fraction of all GW200105 = ', np.sum(w[(mask_spinBig==1) & (maskGW200105==1)])/np.sum(w[maskGW200105]) )
    
    
    
    return 




print_spinning_BHNS()



----- OLD OLD AND WRONG my fault analytical fit Debatri ----- 
--- small >0.05 BH spin ---
fraction of all BHNS =  0.001749085812432054
fraction of all GW200115 =  0.0
fraction of all GW200105 =  0.0

--- moderate >0.2 BH spin---
fraction of all BHNS =  0.0016508440466334352
fraction of all GW200115 =  0.0
fraction of all GW200105 =  0.0

----- old analytical fit Debatri ----- 
--- small >0.05 BH spin ---
fraction of all BHNS =  0.001749085812432054
fraction of all GW200115 =  0.0
fraction of all GW200105 =  0.0

--- moderate >0.2 BH spin---
fraction of all BHNS =  0.0016723922876643346
fraction of all GW200115 =  0.0
fraction of all GW200105 =  0.0

------ Bavera 2021 ------- 

3049458 3049458 3049458
[-0.10094671 -1.4532821  -0.1080839  ... -0.08968286 -0.11026179
 -0.09100559] [-0.06617286 -2.38929258 -0.073884   ... -0.05469238 -0.076303
 -0.05599599]
[ 2.1992  19.4211   2.46888 ...  1.7349   2.54787  1.79218] solMass
[0.60331384 0.13943964 0.29412759 ... 0.63328142 0.34674038 6.27